# Feature Film, Released between 1990-01-01 and 2022-12-31, Rating Count at least 5,000, United States (Sorted by IMDb Rating Descending)
## Data Scraping

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
from random import randint

headers = {"Accept-Language": "en-US,en;q=0.5"}

titles = []
years = []
runtimes = []
genres = []
ratings = []
metascores = []
votes = []
grosses = []
directors = []
casts = []

#Feature Film, Released between 1990-01-01 and 2022-12-31, Rating Count at least 5,000, United States (Sorted by IMDb Rating Descending)

for i in range(1, 7501, 250):
    url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,2022-12-31&num_votes=5000,&countries=us&sort=user_rating,desc&count=250&start='+ str(i) +'&ref_=adv_nxt'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')

    sleep(randint(2,8))

    for container in movie_div:
        title = container.h3.a.text
        titles.append(title)

        year = container.h3.find('span', class_='lister-item-year').text.replace('(', '').replace(')', '')
        years.append(year)

        time = container.p.find('span', class_='runtime').text.strip().replace(' min', '') if container.p.find('span', class_='runtime') else None
        runtimes.append(time)        # runtime in mins

        genre = container.p.find('span', class_='genre').text.strip() if container.p.find('span', class_='genre') else None
        genres.append(genre)

        imdb = float(container.strong.text)
        ratings.append(imdb)

        metascore = container.find('span', class_='metascore').text.strip() if container.find('span', class_='metascore') else None
        metascores.append(metascore)

        nv = container.find_all('span', attrs={'name': 'nv'})

        vote = nv[0].text
        votes.append(vote)

        gross = nv[1].text if len(nv) > 1 else None
        grosses.append(gross)

        director = container.find('p', class_="").text.split('|')[0].replace('\n', '').split(':')[1] if container.find('p', class_="") else None
        directors.append(director)

        try:
          cast = container.find('p', class_="").text.split('|')[1].replace('\n', '').split(':')[1]
        except:
          cast = None
        casts.append(cast)

movies = pd.DataFrame({
    'movie': titles,
    'year': years,
    'runtime(mins)': runtimes,
    'genres': genres,
    'rating': ratings,
    'metascore': metascores,
    'votes': votes,
    'gross($M)': grosses,
    'director(s)': directors,
    'casts': casts
})

KeyboardInterrupt: 

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7345 entries, 0 to 7344
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie          7345 non-null   object 
 1   year           7345 non-null   int64  
 2   runtime(mins)  7345 non-null   int64  
 3   genres         7345 non-null   object 
 4   rating         7345 non-null   float64
 5   metascore      6299 non-null   float64
 6   votes          7345 non-null   int64  
 7   gross($M)      5589 non-null   float64
 8   director(s)    7345 non-null   object 
 9   casts          7344 non-null   object 
dtypes: float64(3), int64(3), object(4)
memory usage: 574.0+ KB


In [ ]:
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies['runtime(mins)'] = movies['runtime(mins)'].astype(int)
movies['votes'] = movies['votes'].str.replace(',', '').astype(int)
movies['metascore'] = pd.to_numeric(movies['metascore'])
movies['gross($M)'] = movies['gross($M)'].str.extract('(\d+.\d+)').astype(float)

In [ ]:
movies

NameError: name 'movies' is not defined

In [ ]:
movies.to_csv('movies.csv', index=False)